# 0. Load Packages and Data File

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [71]:
users = pd.read_csv('users.csv')
brands = pd.read_csv('brands.csv')
receipts = pd.read_csv('receipts.csv')
receipts_item = pd.read_csv('receipts_item.csv')

# 1. Data Quality Issue - Incomplete Information

## 1.1 General Missing Data Info

In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212 entries, 0 to 211
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            212 non-null    object
 1   active        212 non-null    bool  
 2   createdDate   212 non-null    object
 3   lastLogin     172 non-null    object
 4   role          212 non-null    object
 5   signUpSource  207 non-null    object
 6   state         206 non-null    object
dtypes: bool(1), object(6)
memory usage: 10.3+ KB


In [4]:
brands.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1167 non-null   object
 1   barcode       1167 non-null   int64 
 2   brandCode     898 non-null    object
 3   category      1012 non-null   object
 4   categoryCode  517 non-null    object
 5   cpg           1167 non-null   object
 6   name          1167 non-null   object
 7   topBrand      555 non-null    object
dtypes: int64(1), object(7)
memory usage: 73.1+ KB


In [5]:
receipts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119 entries, 0 to 1118
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       1119 non-null   object 
 1   bonusPointsEarned        544 non-null    float64
 2   bonusPointsEarnedReason  544 non-null    object 
 3   createDate               1119 non-null   object 
 4   dateScanned              1119 non-null   object 
 5   finishedDate             568 non-null    object 
 6   modifyDate               1119 non-null   object 
 7   pointsAwardedDate        537 non-null    object 
 8   pointsEarned             609 non-null    float64
 9   purchaseDate             671 non-null    object 
 10  purchasedItemCount       635 non-null    float64
 11  rewardsReceiptStatus     1119 non-null   object 
 12  totalSpent               684 non-null    float64
 13  userId                   1119 non-null   object 
dtypes: float64(4), object(10

In [6]:
receipts_item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6941 entries, 0 to 6940
Data columns (total 36 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   id                                  6941 non-null   int64  
 1   barcode                             3090 non-null   object 
 2   description                         6560 non-null   object 
 3   finalPrice                          6767 non-null   float64
 4   itemPrice                           6767 non-null   float64
 5   needsFetchReview                    813 non-null    object 
 6   partnerItemId                       6941 non-null   int64  
 7   preventTargetGapPoints              358 non-null    object 
 8   quantityPurchased                   6767 non-null   float64
 9   userFlaggedBarcode                  337 non-null    float64
 10  userFlaggedNewItem                  323 non-null    object 
 11  userFlaggedPrice                    299 non

## 1.2 The Consequence of Missing Data
1. We cannot fully present the fact of data. For example, we cannot get the true rank of brands by receipts scanned for most recent month. Some brand names are missing.    
2. When I tried to analyze how many brands were scanned by receipts for each year and month, the missing data cause wrong result. 

In [28]:
t = pd.merge(receipts, receipts_item, left_on=['id'], right_on=['receipt_id'], how='right')
tt = pd.merge(t, brands, left_on=['brandCode'], right_on=['brandCode'], how='left')

In [32]:
tt['month'] = pd.DatetimeIndex(tt['dateScanned']).month
tt['year'] = pd.DatetimeIndex(tt['dateScanned']).year

In [33]:
# many receipts_item missing brandCode information 
tt.isna().sum()

id_x                                        0
bonusPointsEarned                      187491
bonusPointsEarnedReason                187491
createDate                                  0
dateScanned                                 0
finishedDate                           195274
modifyDate                                  0
pointsAwardedDate                      163539
pointsEarned_x                         119146
purchaseDate                             5649
purchasedItemCount                      12913
rewardsReceiptStatus                        0
totalSpent                                  0
userId                                      0
id_y                                        0
barcode_x                              807047
description                            102489
finalPrice                              46806
itemPrice                               46806
needsFetchReview                      1015690
partnerItemId                               0
preventTargetGapPoints            

In [34]:
tt.groupby(['year','month']).agg({'name':'count'})

name
year month         
2020 10        4304
     11       14526
2021 1      1093043
     2        49497
     3         6994

Now we assume that we want to analyze the top 5 brands by receipts scanned for each month.  We need to drop na of brands table before we join it with other tables. 

In [42]:
brands.isna().sum()

id                0
barcode           0
brandCode       269
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
dtype: int64

Since brandCode is the primary key to link receipts_item table, it shouldn't be na. Therefore, we drop rows if its brandCode is null value.  

In [43]:
brands2 = brands.dropna(subset=['brandCode'])

In [44]:
t2 = pd.merge(receipts, receipts_item, left_on=['id'], right_on=['receipt_id'], how='right')
tt2 = pd.merge(t, brands2, left_on=['brandCode'], right_on=['brandCode'], how='left')

In [47]:
tt2['month'] = pd.DatetimeIndex(tt2['dateScanned']).month
tt2['year'] = pd.DatetimeIndex(tt2['dateScanned']).year

In [48]:
tt2.isna().sum()

id_x                                     0
bonusPointsEarned                      963
bonusPointsEarnedReason                963
createDate                               0
dateScanned                              0
finishedDate                           974
modifyDate                               0
pointsAwardedDate                      863
pointsEarned_x                         690
purchaseDate                            21
purchasedItemCount                      49
rewardsReceiptStatus                     0
totalSpent                               0
userId                                   0
id_y                                     0
barcode_x                             3851
description                            381
finalPrice                             174
itemPrice                              174
needsFetchReview                      6134
partnerItemId                            0
preventTargetGapPoints                6589
quantityPurchased                      174
userFlagged

In [49]:
tt2.isna().sum()

id_x                                     0
bonusPointsEarned                      963
bonusPointsEarnedReason                963
createDate                               0
dateScanned                              0
finishedDate                           974
modifyDate                               0
pointsAwardedDate                      863
pointsEarned_x                         690
purchaseDate                            21
purchasedItemCount                      49
rewardsReceiptStatus                     0
totalSpent                               0
userId                                   0
id_y                                     0
barcode_x                             3851
description                            381
finalPrice                             174
itemPrice                              174
needsFetchReview                      6134
partnerItemId                            0
preventTargetGapPoints                6589
quantityPurchased                      174
userFlagged

In [50]:
tt2.groupby(['year','month']).agg({'name':'count'})

name
year month      
2020 10        0
     11        0
2021 1       634
     2         1
     3         0

We can see the result is very different from before dropping null value in brands table.

# 2. Data Quality Issue - Duplicate Data

1. There are some brandCode are duplicated such as HUGGIES and GOODNITES.   
2. Their topBrand status are different even if they have the same brandCode.    
3. Solution: Delete duplicate entry based on right informtaion. 

In [85]:
brands[brands.groupby(['brandCode'])['id'].transform('count') > 1]

,id,barcode,brandCode,category,categoryCode,cpg,name,topBrand
628,5bd2011f90fa074576779a17,511111704652,HUGGIES,Baby,NaN,"DBRef('Cogs', ObjectId('550b2565e4b001d5e9e414...",Huggies,False
1036,5db32879ee7f2d6de4248976,511111112938,GOODNITES,Baby,BABY,"DBRef('Cogs', ObjectId('55b62995e4b0d8e685c142...",GoodNites,True
1074,5c7d9cb395144c337a3cbfbb,511111707202,HUGGIES,Baby,BABY,"DBRef('Cogs', ObjectId('5459429be4b0bfcb1e8640...",Huggies,True
1079,5bd200fc965c7d66d92731eb,511111204640,GOODNITES,Baby,NaN,"DBRef('Cogs', ObjectId('550b2565e4b001d5e9e414...",Goodnites,False


# 3. Data Quality Issue - Inaccurate Data

1. Some brandCodes have different names and different topBrand. For example, such as GOODNITES has two names under Goodnites and GoofNites, and one is topBrand and another one is not topBrand. 

In [67]:
brands[brands["brandCode"]=="GOODNITES"]

,id,barcode,brandCode,category,categoryCode,cpg,name,topBrand
1036,5db32879ee7f2d6de4248976,5.111111e+11,GOODNITES,Baby,BABY,"DBRef('Cogs', ObjectId('55b62995e4b0d8e685c142...",GoodNites,True
1079,5bd200fc965c7d66d92731eb,5.111112e+11,GOODNITES,Baby,NaN,"DBRef('Cogs', ObjectId('550b2565e4b001d5e9e414...",Goodnites,False


In [96]:
brands[brands["brandCode"]=="HUGGIES"]

,id,barcode,brandCode,category,categoryCode,cpg,name,topBrand
628,5bd2011f90fa074576779a17,511111704652,HUGGIES,Baby,NaN,"DBRef('Cogs', ObjectId('550b2565e4b001d5e9e414...",Huggies,False
1074,5c7d9cb395144c337a3cbfbb,511111707202,HUGGIES,Baby,BABY,"DBRef('Cogs', ObjectId('5459429be4b0bfcb1e8640...",Huggies,True
